In [1]:
import torch as th 
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from model import Generator, Discriminator, gradient_penalty

In [ ]:
""" 
Import Data
"""


In [ ]:
""" 
Hyperparameters
"""

# Optimizer params
g_lr = 0.001 
d_lr = 0.001
b1 = 0.5 
b2 = 0.999  

# WGAN params
N_critic = 5            # nr of times to train discriminator more
lambda_gp = 10          # gradient penalty hyperpraram

# Training params
MAX_EPOCHS = 500

In [ ]:
""" 
Model Definitions
"""
generator = Generator()
discriminator = Discriminator()


# Optimizers
optimizer_G = th.optim.Adam(generator.parameters(), lr=g_lr, betas=(b1, b2)) 
optimizer_D = th.optim.Adam(discriminator.parameters(), lr=d_lr, betas=(b1, b2))

In [ ]:
# Main training loop
def train(generator, discriminator, optimizer_g, optimizer_d, data_loader):
    for epoch in range(MAX_EPOCHS):
        # real == batch (confusing naming I know...)
        for real in data_loader:
            # Create new data object with noise and same edge_index 
            for _ in range(N_critic):
                noise_batch = []

                # Create n_batch number of noise_vectors and batch it
                # We must create a noise vector with the corresponding graph in the actual data
                for batch_data in real:
                    noise = th.randn(batch_data.x.shape)
                    noise = Data(x=noise, edge_index=batch_data.edge_indices)
                    noise_batch.append(noise)

                noise_batch = Batch.from_data_list(noise_batch)

                # Input noise_data into generator
                global fake 
                fake = Generator(noise_batch)
                discriminator_fake = discriminator(fake).reshape(-1)        # discriminator scores for fakes
                discriminator_real = discriminator(real).reshape(-1)        # discriminator scores for reals
                gp = gradient_penalty(discriminator, real, fake)

                # Discriminator loss and train
                loss_discriminator = -(th.mean(discriminator_real) - th.mean(discriminator(fake))) + lambda_gp * gp
                discriminator.zero_grad() 
                loss_discriminator.backward() 
                optimizer_d.step()

            # Generator loss and train
            output = discriminator(fake).reshape(-1)        # discriminator scores for fake
            loss_generator = -th.mean(output)               # loss for genereator = the discriminators' judgement
                                                            # higher score = better

            generator.zero_grad()
            loss_generator.backward()
            optimizer_g.step()
    
        # TODO: Evaluation and logging code??